# Diffusion anisotropy

## Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from cage_data import cage1_info
from fishmol.utils import update_progress, vector, Arrow3D
from fishmol import msd

## Load water CoM data

In [2]:
water_com = pd.read_excel("test/cage1-500K-water-com.xlsx", header=0, index_col=0, engine = "openpyxl")
water_com.head()

,water1_x,water1_y,water1_z,water2_x,water2_y,water2_z,water3_x,water3_y,water3_z,water4_x,...,water5_z,water6_x,water6_y,water6_z,water7_x,water7_y,water7_z,water8_x,water8_y,water8_z
0,10.586316,19.902134,10.380009,0.624155,10.923925,8.298079,-1.967392,4.261146,12.241481,4.693672,...,4.725172,15.090375,8.649502,6.883228,17.759366,15.321196,2.865725,11.019290,3.578953,0.703805
1,10.597616,19.945151,10.393126,0.617881,10.942585,8.312331,-1.961926,4.253910,12.205821,4.689873,...,4.745156,15.076014,8.640429,6.849210,17.749958,15.278667,2.889409,11.049146,3.587807,0.663123
2,10.610208,20.003821,10.408405,0.610859,10.967922,8.330438,-1.953047,4.243309,12.156421,4.687198,...,4.767198,15.058083,8.628816,6.804086,17.738720,15.225162,2.920655,11.086905,3.600664,0.608757
3,10.622718,20.062112,10.416761,0.604688,10.993416,8.349495,-1.945208,4.229969,12.106099,4.687512,...,4.786539,15.040753,8.617851,6.760128,17.728982,15.181476,2.948253,11.122864,3.614650,0.558271
4,10.636664,20.122749,10.416813,0.597829,11.020753,8.373153,-1.941190,4.211549,12.054352,4.691621,...,4.808036,15.020218,8.605213,6.714651,17.717884,15.151479,2.970079,11.155729,3.629731,0.511643


In [3]:
def proj_d(df, num, start_idx = 0, n_theta = 120, n_phi = 120, filename = None, plot = False, figname =None):
    """
    Calculates the MSD and D alogn the direction specified by vec.
    """
    # A spherical mesh of vectors
    phi = np.linspace(-np.pi, np.pi, num = n_phi)
    theta = np.linspace(-np.pi/2, np.pi, num = n_theta)

    phi, theta = np.meshgrid(phi, theta)
    
    x = np.sin(phi) * np.sin(theta)
    y = np.sin(phi) * np.cos(theta)
    z = np.cos(phi)
    
    vecs = np.stack((x.flatten(), y.flatten(), z.flatten()), axis = 1) 
    ave_D = np.zeros(len(vecs))
    
    t0 = start_idx * 5
    t_end = t0 + 5*(len(df)-1) # 5 fs is the interval of traj
    t = np.linspace(t0, t_end, num = len(df))
    t = t[1000:] - 5000
    
    for i, vec in enumerate(vecs):
        msd_df = np.zeros((len(df), num))
        for j in range(num):
            temp = np.array(df.iloc[:, 3*j:3*j+3])
            temp = msd.traj_proj(temp, vec)
            msds = msd.msd_fft(temp.reshape(len(temp),1))
            msd_df[:, j] = msds

        ave_msd = msd_df.mean(axis=1)[1000:]
        
        linear_model=np.polyfit(t/1000, ave_msd, 1)
        ave_D[i] = linear_model[0]/20000
        update_progress(i / len(vecs))
    update_progress(1)
    return vecs, ave_D

In [ ]:
water_num = 8
vecs, ave_d = proj_d(df = water_com, num = water_num)

Progress: [■■■○○○○○○○○○○○○○○○○○] 15.4%


In [ ]:
results = pd.DataFrame()
results["x"] = vecs[:,0]
results["y"] = vecs[:,1]
results["z"] = vecs[:,2]
# results[["x", "y", "z"]] = vecs
results["mean_D"] = ave_d
results.to_excel("test/cage1-500K-aniso.xlsx")

In [ ]:
locs = [diag.array/40000 for diag in diags]

In [ ]:
fig = plt.figure(figsize = (5, 4.6))
ax = fig.add_axes([0.0,0.04,0.90,0.96], projection='3d')

# ax.plot_trisurf(x, y, z, edgecolor ='none', cmap='PuBu', alpha=0.8)
ax.scatter3D(results["x"], results["y"], results["z"], edgecolor ='none', marker = ".", color = "#08519c", alpha=0.3)

for i, axis in enumerate(diags):
    axis = axis.array/40000
    a = Arrow3D([0, axis[0]], [0, axis[1]], [0, axis[2]], mutation_scale=15, 
            lw=1, arrowstyle="-|>", color="#252525")
    ax.add_artist(a)
    ax.text(*locs[i], labels[i], zdirs[i])

idx = np.where(ave_d == np.amax(ave_d))
h_path = vecs[idx]*ave_d[idx]

a = Arrow3D(*zip(np.zeros(3), h_path[0]), mutation_scale=15, 
        lw=1, arrowstyle="-|>", color="#b2182b")
ax.add_artist(a)
ax.text(*h_path[0], "D$_{max}$", None)

ax.set_xlabel("$D_x$")
ax.set_ylabel("$D_y$")
ax.set_zlabel("$D_z$")

ax.ticklabel_format(axis='both', style='sci', scilimits=[-4,4], useMathText=True)

plt.savefig("test/cage1-aniso-500K.jpg", dpi = 600)

plt.show()